# Group Project Outline

## Objective: 
### Simulate one portfolio composed with top stocks in top industries (growth-wise), using historical stock return data, and measure its performance in aspects of the Sharp Ratio of the portfolio. 

## Steps: 
### Use historical stock price data to: 
#### 1. Compute one-month average returns of all the stocks in S&P 500 (say we use the historical 25 years data, then we have 25 x 12 = 300 monthly average returns on one stock. We don't want to use daily returns because thay may be too much data).

#### 2. Use the average returns to do 500 regression models and find the slope of the regression line and store those slopes in one dataframe. By now, we have one corresponding number for one stock that represents the growth of the stock, higher the slope higher the return. 

#### 3. Catagorize the slope data we have into sectors (industries), and we can calculate the average of those slope numbers in each industry. This average may represent the overall trend of the slope (don't know if this make sense mathematically though). 

#### 4. Find the top five industries that have the highest average and build a correlation table. See if one industries growth has something to do with another industry's growth (we can also pick the industries that we think should have close correlation). 

#### 5. For the picked-up industries in #4, we pick five stocks that has the highest regression slope number and do a correlation table based on their historical return averages that we calculated in #1. 

#### 6. If possible, we can use the historical returns of those five stocks and calculate the CAPM beta between the selected stocks and the corresponsing industry (here, we can look at the homeword we did on CAPM beta, in that homework we calculated the beta of one stock to the whole market, but now we focus on the beta between the stock and its industry. Shouldn't be too hard cuz we already have the code). 

#### 7. If possible, we can use the historical returns of selected industries and calculate the CAPM beta between the selected industry to the whole market.

#### 8. Simulate a portfolio with the top stocks in each sector and calculate the Sharp Ratio using their historical returns. (This shouldn't be hard cuz Sharp Ratio is one simple formula.)

## Results
#### We will discover the highest return industry, its correlation with other industries, and its beta with the market. 
#### We will discover the hottest stock in the top industries and their beta with their corresponding industries. 
#### We simulated one portofolio with stocks we picked up based on the regression slope (which represent the growth of the stock) and we will have different betas (between the stock and the industry, and between the industries and the market), and the Sharp Ratio of the portfolio. 

## Methods used: 
#### Regression (whish means visualization)
#### Functions
#### Class 
#### Maybe unittest to test classes 
#### Dataframe operations 
#### ...

## Need to decide: 
#### Steps needed 
#### Methods needed
#### If the math in the steps make sense
#### Rephrase the Steps to make them clearer and more instructive for operational purposes
#### ...


In [16]:
import sys
!{sys.executable} -m pip install --upgrade pandas --user

# Import Packages Needed

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pdb
import math
import scipy.stats as sp
import statsmodels.api as sm
import datetime as dt

# Clean Data

In [3]:
df = pd.read_csv('financials.csv')
df1 = df['Symbol']

In [5]:
for i in df1: 
    myticker = yf.Ticker(i)
    history = myticker.history()

- AET: No data found for this date range, symbol may be delisted
- ALXN: No data found, symbol may be delisted
- AGN: No data found, symbol may be delisted
- APC: No data found, symbol may be delisted
- ANDV: No data found for this date range, symbol may be delisted
- BHGE: No data found, symbol may be delisted
- BBT: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- CA: No data found for this date range, symbol may be delisted
- CBG: No data found for this date range, symbol may be delisted
- CBS: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- CTL: No data found, symbol may be delisted
- XEC: No data found for this date range, symbol may be delisted
- CXO: No data found, symbol may be delisted
- CSRA: No data found for this date range, symbol may be delisted
- DWDP: No data found, symbol may be delisted
- DPS: None
- ETFC: No data found, symbol 

In [4]:
missing = ('AET', 'ALXN', 'AGN', 'ARNC', 'APC','ANDV','BHGE','BBT','BRK.B','BF.B','CHK','CA','CBG','CBS','CELG','CTL','XEC','CXO','CSRA','DWDP','DPS','ETFC','EVHC','ESRX','FLIR','GGP','HRS','HCP','JEC','LB','LUK','KORS','MYL','NFX','NBL','PCLN','RTN','RHT','COL','SCG','SNI','STI','SYMC','TIF','TWX','TMK','TSS','UTX','VAR','VIAB','HCN','WYN')

In [5]:
df = df.set_index('Symbol')
for i in missing:
    df=df.drop(index=i)
sp500 = df.reset_index()

In [6]:
tickers = sp500['Symbol']
industries = sp500['Sector'].unique()
df_industry = pd.DataFrame(industries)
df_industry.columns = (['Industries'])

In [7]:
#cleaning for S&P500
sp500index = pd.read_csv('S&P 500 Historical Data.csv')
sp500index.head()

sp500index = sp500index[["Date", "Price"]]
sp500index.head(100)
#sp500index['date'] = pd.to_datetime(df['date'])
sp500index = sp500index.sort_index(ascending=False, axis=0)
#sp500index["Date"] = sp500index["Date"].values[::-1]
sp500index.head(100)

sp500indexdf = pd.DataFrame(sp500index)
#sp500indexdf.dtypes
sp500indexdf['Price'] = sp500indexdf['Price'].apply(lambda x: float(x.split()[0].replace(',', '')))
sp500indexdf = sp500indexdf.drop(columns=['Date'])
sp500indexdf.reset_index(drop=True, inplace=True)
sp500indexdf

,Price
0,1394.46
1,1366.42
2,1498.58
3,1452.43
4,1420.60
...,...
247,3500.31
248,3363.00
249,3269.96
250,3621.63


In [8]:
#importing industry
industrydata = pd.read_csv('all_indusry_index.csv')
industrydata.head()

,Date,Industrials,Health Care,Information Technology,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Consumer Staples,Energy,Telecommunication Services
0,2000-01,77.839608,31.436890,42.091193,36.773550,14.089150,135.541414,17.903646,19.922140,25.802876,16.894283,16.219559
1,2000-02,75.739446,37.561319,46.145104,36.896504,14.231424,144.283133,16.261889,23.765975,24.356122,16.414776,15.087821
2,2000-03,85.186907,36.331098,52.616795,32.007439,13.948501,135.724248,16.209372,29.776267,21.426221,17.803222,16.625273
3,2000-04,93.109711,29.284575,43.837446,30.498866,13.833688,147.999373,17.263157,22.829001,23.481253,17.712147,17.551001
4,2000-05,83.399070,29.521381,39.646895,28.003616,17.867399,156.479204,17.469020,22.816852,23.845739,18.122205,16.369433


# Stock Operation Class

In [9]:
class Stock:
    def __init__ (self, ticker):
        self.ticker = ticker
        self.industry = sp500['Sector'][sp500[sp500['Symbol'] == self.ticker].index.tolist()].tolist()[0]
    def monthly_avg(self):
        history = yf.Ticker(self.ticker).history(start='2000-01-01', end='2020-12-31').reset_index()
        monthly_avg = history.groupby(pd.PeriodIndex(history['Date'], freq="M"))['Close'].mean()
        monthly_avg = pd.DataFrame(monthly_avg)
        monthly_avg.columns = ([self.ticker])
        return monthly_avg
    def plot(self):
        data = self.monthly_avg()
        plot = data.plot()
    def reg_slope(self):
        data = self.monthly_avg()
        data = data.reset_index()
        data['Month'] = range(1,len(data)+1)
        X = data['Month']
        y = data[self.ticker]
        model = sm.OLS(y, X).fit()
        predictions = model.predict(X)
        slope = model.params.values
        return slope
    def price_change(self):
        data = pd.DataFrame(self.monthly_avg())
        data = data.reset_index()
        begin = data[self.ticker][data[data['Date'] == '2000-01'].index.tolist()].tolist()
        end = data[self.ticker][data[data['Date'] == '2020-12'].index.tolist()].tolist()
        price_change = (((end[0]-begin[0]))/begin[0])
        return price_change
    def market_cap(self):
        history = yf.Ticker(self.ticker).history(start='2000-01-01', end='2020-12-31').reset_index()
        volume = history.groupby(pd.PeriodIndex(history['Date'], freq="M"))['Volume'].mean()
        price = self.monthly_avg()
        df_v = pd.DataFrame(volume)
        df_p = pd.DataFrame(price)
        market_cap = pd.concat([df_p, df_v], axis=1)
        market_cap['Market_Cap'] = market_cap['Volume'] * market_cap[self.ticker]
        market_cap.columns = ([self.ticker + '_' + 'Price'] , [self.ticker + '_' + 'Volume'] , [self.ticker + '_' + 'Market_Cap'])
        return market_cap

In [8]:
aapl = Stock('AAPL')

In [9]:
str(type(aapl.monthly_avg()))

"<class 'pandas.core.frame.DataFrame'>"

In [21]:
import unittest

class TestStock_CAPM(unittest.TestCase):
    aapl = Stock('AAPL')
    apple = CAPM('AAPL')
    applreturn = apple.CAPM_stock_industry(apple.stock)
    apple_ind_return = apple.CAPM_industry(apple.industry)
    sp500return = apple.CAPM_market(apple.market)
    def test_monthly_avg(self):
        avg = self.aapl.monthly_avg()
        self.assertEqual(str(type(avg)), "<class 'pandas.core.frame.DataFrame'>")
    
    def test_price_change(self):
        change = self.aapl.price_change()
        self.assertEqual(round(change, 4), 158.5699)
    
    def test_reg_slope(self):
        slope = self.aapl.reg_slope()
        self.assertEqual(round(slope[0], 4), 0.1694)
        
    def test_market_cap(self):
        avg = self.aapl.monthly_avg()
        self.assertEqual(str(type(avg)), "<class 'pandas.core.frame.DataFrame'>")
        
    def test_capm_stock_indus(self):
        beta = self.apple.beta_sensitivity_stock_industry(applreturn, apple_ind_return)
        self.assertEqual(round(beta[0], 8), 0.82676916)
    def test_capm_indus_market(self):
        beta = self.apple.beta_sensitivity_industry_market(apple_ind_return, sp500return)
        self.assertEqual(round(beta[0], 8), 0.96532034)

unittest.main(argv=[''], verbosity=2, exit=False)


test_capm_indus_market (__main__.TestStock_CAPM) ... ok
test_capm_stock_indus (__main__.TestStock_CAPM) ... ok
test_market_cap (__main__.TestStock_CAPM) ... ok
test_monthly_avg (__main__.TestStock_CAPM) ... ok
test_price_change (__main__.TestStock_CAPM) ... ok
test_reg_slope (__main__.TestStock_CAPM) ... ok

----------------------------------------------------------------------
Ran 6 tests in 1.389s

OK


# S&P Operation Class

In [10]:
class SP500_Operation:
    def __init__ (self, df):
        self.df = df
    def stock_industry_price_change(self):
        data = []
        for i in tickers[:10]:
            try:
                stock = Stock(i)
                data.append((stock.ticker,stock.industry,stock.price_change()))
            except:
                pass
        stock_industry_price_change = pd.DataFrame(data)
        stock_industry_price_change.columns = ['Symbol', 'Industry','Price_Change']
        return stock_industry_price_change
    def stocks_and_returns_in_industry(self, industry):
        df = self.stock_industry_price_change()
        data = df[df['Industry']== industry]
        return data
    def industry_index(self, industry):
        A = Industry_Operation(df_industry)
        industry_market_cap, names, df = A.total_market_cap(industry)
        weight = A.stock_weight_in_industry(industry)
        #data = pd.DataFrame()
        data_list = []
        for i in names: 
            stock = Stock(i)
            price = stock.monthly_avg()
            #data[i] = price
            data_list.append(price)
        data = pd.concat(data_list, axis=1)
        a = pd.DataFrame(data.values*weight.values, columns=data.columns, index=data.index)
        Industry_Index = pd.DataFrame(a.sum(axis = 1))
        Industry_Index.columns = ([industry])
        return Industry_Index
    def top_stocks_in_industry_index(self, industry, number_stocks=0):
        A = Industry_Operation(df_industry)
        industry_market_cap, names, df = A.total_market_cap(industry)
        data = []
        for i in names:
            try:
                stock = Stock(i)
                data.append((stock.ticker,stock.industry,stock.price_change()))
            except:
                pass
        df = pd.DataFrame(data)
        df.columns = ['Symbol', 'Industry','Price_Change']
        top_stocks_in_industry_index = df.nlargest(number_stocks,'Price_Change')
        return top_stocks_in_industry_index

In [11]:
SP500 = SP500_Operation(sp500)

In [16]:
Industry_Index = SP500.industry_index('Real Estate')

# Industry Operation Class

In [11]:
class Industry_Operation:
    def __init__ (self, df):
        self.df = df
    def total_market_cap(self, industry):
        stocks = sp500['Symbol'][sp500['Sector'] == industry]
        aapl = Stock('AAPL')
        df = pd.DataFrame(aapl.monthly_avg()).drop(['AAPL'], axis=1) 
        df.reset_index()
        data = pd.DataFrame()
        for i in stocks:
            stock = Stock(i)
            a = stock.market_cap()
            b = a.iloc[:,[2]]
            df = df.merge(b, how='left', on='Date')
            df = df.dropna(axis=1)
            data[industry + ' ' + 'Total Market Cap'] = df.sum(axis=1, numeric_only=True)
        industry_market_cap = pd.DataFrame(data[industry + ' ' + 'Total Market Cap'])
        columns_names = df.columns
        names = []
        for item in columns_names:
            name = item[0]
            tickername = name.split('_')[0]
            names.append(tickername)
        return industry_market_cap, names, df
    def stock_weight_in_industry(self, industry):
        industry_market_cap, names, df = self.total_market_cap(industry)
        weight = df.div(industry_market_cap.iloc[:,0], axis=0)
        return weight 
    def plot(self, industry):
        A = SP500_Operation(sp500)
        data = A.industry_index(industry)
        plot = data.plot()
    def price_change(self, industry):
        A = SP500_Operation(sp500)
        data = A.industry_index(industry)
        data = data.reset_index()
        begin = data[industry][data[data['Date'] == '2000-01'].index.tolist()].tolist()
        end = data[industry][data[data['Date'] == '2020-12'].index.tolist()].tolist()
        price_change = (((end[0]-begin[0]))/begin[0])
        return price_change
    def all_industry_index(self):
        industries = self.df
        A = SP500_Operation(sp500)
        #df = pd.DataFrame()
        df_list = []
        for i in industries['Industries']:
            data = A.industry_index(i)
            #df[i] = data
            df_list = data
        df = pd.concat(df_list, axis=1)
        return df

In [17]:
A = Industry_Operation(df_industry)

In [18]:
all_indusry_index = A.all_industry_index()

KeyboardInterrupt: 

In [134]:
all_indusry_index

,Industrials,Health Care,Information Technology,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Consumer Staples,Energy,Telecommunication Services
Date,,,,,,,,,,,
2000-01,77.839604,31.436889,42.091191,36.751475,14.089149,135.502717,18.156621,19.922140,25.802876,16.894285,16.219778
2000-02,75.739439,37.561319,46.145104,36.871865,14.231425,144.248952,16.492494,23.765975,24.356123,16.414776,15.088013
2000-03,85.186909,36.331097,52.616795,31.991274,13.948501,135.688905,16.434815,29.776266,21.426220,17.803222,16.625511
2000-04,93.109713,29.284575,43.837446,30.474902,13.833689,147.961464,17.494763,22.829001,23.481252,17.712147,17.551257
2000-05,83.399077,29.521381,39.646896,27.981981,17.867398,156.417924,17.689196,22.816852,23.845741,18.122206,16.369677
...,...,...,...,...,...,...,...,...,...,...,...
2020-08,162.008984,186.979917,131.345997,1661.478352,61.196670,94.784680,118.372085,114.206887,124.415587,40.135760,40.119231
2020-09,161.725885,186.534894,135.358235,1822.817043,61.266695,94.650005,124.018530,121.885670,122.457478,36.615269,40.679937
2020-10,166.240082,196.498513,134.027258,1944.321709,66.192477,94.869431,131.083207,124.187648,124.837200,34.055652,38.799217


In [127]:
#all_indusry_index.to_csv('all_indusry_index.csv')

In [101]:
A.price_change('Real Estate')

5.070165609895448

# CAPM 

 
\begin{aligned} &E[R_i] - R_f  = \beta_i ( E[R_m] - R_f ) \\ \\ &\textbf{where:} \\ &ER_i = \text{expected return of stock i} \\ &R_f = \text{risk-free rate} \\ &\beta_i = \text{beta of the stock} \\ &ER_m - R_f = \text{market risk premium} \\ \end{aligned} 


In [16]:
R_f = 0.1625
class CAPM:
    def __init__(self,stock):
        self.stock = stock
        self.industry = sp500['Sector'][sp500[sp500['Symbol'] == self.stock].index.tolist()].tolist()[0]
        self.market = sp500indexdf
    def CAPM_stock_industry(self, stock):
        #this is stock monthly percent change
        s = Stock(stock)
        a = s.monthly_avg()
        a = pd.DataFrame(a)
        a.columns=([stock])
        pch = a.pct_change()
        pcch = pch-R_f
        pcch = pcch.iloc[1:]
        pcch.reset_index(drop=True, inplace=True)
        #pcch = pcch.drop('Date', axis=1, inplace=True)
        return pcch
    def CAPM_industry(self, industry):
        indus = sp500['Sector'][sp500[sp500['Symbol'] == self.stock].index.tolist()].tolist()[0]
        i = industrydata
        i = i[indus]
        idata = pd.DataFrame(i)
        idata = idata.pct_change()
        idata = idata - R_f
        idata = idata.iloc[1:]
        idata.reset_index(drop=True, inplace=True)
        return idata
    def CAPM_market(self, market):
        #This returns market return - R_f
        sp500_monthly_return = sp500indexdf['Price'].pct_change()
        sp500_monthly_return = sp500_monthly_return-R_f
        sp500_monthly_return = sp500_monthly_return.iloc[1:]
        sp500_monthly_return.reset_index(drop=True, inplace=True)
        return sp500_monthly_return
    def beta_sensitivity_stock_industry(self, ticker_rtn, indus_rtn):
        ticker_m_rtn = np.array(ticker_rtn)
        indus_m_rtn = np.array(indus_rtn)
        ind_rtn_trans = np.transpose(indus_m_rtn)
        ind_rtn_mul = np.matmul(ind_rtn_trans, indus_m_rtn)
        ind_inv = np.reciprocal(ind_rtn_mul)
        ind_inv_mul_appl = np.matmul(ind_rtn_trans, ticker_m_rtn)
        betaEst = np.multiply(ind_inv, ind_inv_mul_appl)
        return betaEst[0]
    def beta_sensitivity_industry_market(self, indus_rtn, market_rtn):
        indus_m_rtn = np.array(indus_rtn)
        market_m_rtn = np.array(market_rtn)
        sp500_rtn_trans = np.transpose(market_m_rtn)
        sp500_rtn_mul = np.matmul(sp500_rtn_trans, market_m_rtn)
        sp500_inv = np.reciprocal(sp500_rtn_mul)
        sp500_inv_mul_appl = np.matmul(sp500_rtn_trans, indus_m_rtn)
        betaEst = np.multiply(sp500_inv, sp500_inv_mul_appl)
        return betaEst

In [17]:
i = industrydata
u = i['Health Care']
u
u = pd.DataFrame(u)
pchu = u.pct_change()
pcchu = pchu-R_f
pcchu = pcchu.iloc[1:]
pcchu.reset_index(drop=True, inplace=True)
pcchu

,Health Care
0,0.032317
1,-0.195252
2,-0.356453
3,-0.154414
4,-0.044202
...,...
246,-0.138912
247,-0.164880
248,-0.109086
249,-0.148164


In [18]:
test1 = CAPM('AAPL')
test1.CAPM_stock_industry(test1.stock)
#test1.industry
SP500test = SP500_Operation('AAPL')
#SP500test.stocks_and_returns_in_industry('Technology')
#a = test1.monthly_avg()
#a = pd.DataFrame(a)
#a.columns=([stock])
#pch = a.pct_change()
test2 = CAPM('AAPL')
applreturn = test2.CAPM_stock_industry(test2.stock)
print(applreturn)
sp500return = test2.CAPM_market(test2.market)
print(sp500return)
apple_ind_return = test2.CAPM_industry(test2.industry)
print(apple_ind_return)
beta = test2.beta_sensitivity_stock_industry(applreturn, apple_ind_return)
applreturn
sp500return
beta
beta2 = test2.beta_sensitivity_industry_market(apple_ind_return, sp500return)
print("Beta btw ticker and industry: ",beta)
print("Beta btw industry and s&p500: ",beta2)

         AAPL
0   -0.082402
1   -0.011437
2   -0.204503
3   -0.343604
4   -0.188321
..        ...
246  0.066734
247 -0.180721
248 -0.151313
249 -0.157558
250 -0.074765

[251 rows x 1 columns]
0     -0.182608
1     -0.065780
2     -0.193296
3     -0.184415
4     -0.138566
         ...   
246   -0.092435
247   -0.201728
248   -0.190166
249   -0.054954
250   -0.125379
Name: Price, Length: 251, dtype: float64
     Information Technology
0                 -0.066187
1                 -0.022253
2                 -0.329354
3                 -0.258093
4                  0.061642
..                      ...
246               -0.082472
247               -0.131953
248               -0.172333
249               -0.120397
250               -0.099223

[251 rows x 1 columns]
Beta btw ticker and industry:  [0.82676916]
Beta btw industry and s&p500:  [0.96532034]


# Test Area 🎲🎲🎲

### Get the correlation among selected stocks

In [469]:
top_return_symbols = top_price_change['Symbol']

In [473]:
# Setting the index column for other dataframe to concat
aapl = Stock('AAPL')
top_stock_monthly_avg = pd.DataFrame(aapl.monthly_avg()).drop(['Close'], axis=1) 
# Generating the monthly average stock price for the top stocks in one dataframe
for i in top_return_symbols:
    stock = Stock(i)
    stock_i_monthly_avg = pd.DataFrame(stock.monthly_avg())
    stock_i_monthly_avg.columns=([i])
    top_stock_monthly_avg = pd.concat([stock_i_monthly_avg,top_stock_monthly_avg], axis=1)
    
stock_corr = top_stock_monthly_avg.corr() 
stock_corr

,MMM,ABT,AOS,ADBE,ATVI
MMM,1.000000,0.849360,0.983290,0.769558,0.919592
ABT,0.849360,1.000000,0.875211,0.969427,0.919668
AOS,0.983290,0.875211,1.000000,0.818619,0.953318
ADBE,0.769558,0.969427,0.818619,1.000000,0.910988
ATVI,0.919592,0.919668,0.953318,0.910988,1.000000


In [420]:
aapl = Stock('AAPL')
data = aapl.monthly_avg()
#aapl = pd.DataFrame(data)

In [457]:
abt = Stock('ABT')
df = pd.DataFrame(abt.monthly_avg()).columns=(['ABT'])
df

,Close
Date,
2000-01,8.914329
2000-02,9.112125
2000-03,8.911923
2000-04,10.401491
2000-05,10.793253
...,...
2020-08,100.550693
2020-09,103.330695
2020-10,106.110295


In [389]:
mmm = Stock('MMM')
data = mmm.monthly_avg()
#mmm = pd.DataFrame(data)

In [396]:
data = pd.concat([aapl, mmm], axis=1)

In [397]:
data.corr()

,AAPL,MMM
AAPL,1.000000,0.799666
MMM,0.799666,1.000000


In [412]:
aapl

,AAPL
Date,
2000-01,0.791221
2000-02,0.854596
2000-03,0.983694
2000-04,0.942376
2000-05,0.771708
...,...
2020-08,116.342647
2020-09,114.222799
2020-10,115.500651


In [415]:
aapl.drop(['AAPL'],axis=1)

""
Date
2000-01
2000-02
2000-03
2000-04
2000-05
...
2020-08
2020-09
2020-10


# Tasks to be done (this is written before thanskgiving)

1. Write instructions on how to use each class and functions -- Bruce
2. Finish def industry_index in SP500 Operation Class -- Kathy 
   Get top industies. Get top stocks in the top industries. 
3. Finish the CAPM Class -- Joseph 
   Get the beta of the selected stocks (beta bw this stock and its inline industry). Get the beta of the selected industries (bw the market). We need SP500 index numbers (proly in yf)
   Demo:
       def CAPM_stock_industry(self, stock)
           i = Stock(stock)
           industry = i.industry 
       def CAPM_industry_market(self, industry)
       def sensitivity_stock_industry
       def sensitivity_industry_market 
4. Unittest -- Kathy 
5. Regression on the stock and the industry (we look for R^2, which explains how much of the stock price chance is contributed by the price change of the industry) (this concept is similar with beta) -- Bruce
6. Sharpe Ratio of the portfolio -- Bruce
7. Portfolio return graph -- Bruce 

## Code part DONE by 26th Nov. 

# Paper!!!!!!!

### Paper DONE by Nov. 30 

# PPT by Dec. 1